In [1]:
import random
from src.transforms import GrayScaleImage
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from src.models import MINOS
from src.dataset_loaders import (
    ISAdetectDataset,
    random_train_test_split,
)

device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using device: {device}")

num_epochs = 1
learning_rate = 0.001
batch_size = 16
SEED = random.randint(0, 1000)

dataset = ISAdetectDataset(
    dataset_path="../../dataset/ISAdetect/ISAdetect_full_dataset",
    transform=GrayScaleImage(100, 100),
    file_byte_read_limit=100 * 100,
)
train_set, test_set = random_train_test_split(dataset, test_split=0.2, seed=SEED)


train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=1)

Using device: mps


### Set up weights and biases


In [2]:
import wandb

wandb.login()

wandb.init(
    project="master-thesis",
    config={
        "architecture": "MINOS",
        "target_feature": "endianness",
        "optimizer": "Adam",
        "loss_function": "BCELoss",
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "seed": SEED,
    },
)

wandb: Currently logged in as: stianjsu (mikkel-stian-master-team). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


### Define model and optimizer


In [3]:
criterion = nn.BCELoss()
model = MINOS(num_classes=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

wandb.watch(
    model, criterion=nn.BCELoss, log_graph=True, log=["gradients", "parameters", "all"]
)

In [4]:
from tqdm import tqdm

endianness_map = {"little": 0, "big": 1}
print("Training model...")
# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, (batch_x, batch_y) in enumerate(
        tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    ):

        mask = [
            i
            for i, arch in enumerate(batch_y["architecture"])
            if arch not in ["mips", "mipsel"]
        ]
        if not mask:
            continue  # Skip batch if all samples are MIPS

        # Apply mask to batch
        filtered_batch_x = batch_x[mask].to(device)
        filtered_batch_y = {k: [v[i] for i in mask] for k, v in batch_y.items()}

        optimizer.zero_grad()
        output = model(filtered_batch_x / 255.0)
        targets = torch.tensor(
            [endianness_map[e] for e in filtered_batch_y["endianness"]],
            dtype=torch.float32,
        ).to(device)
        loss = criterion(output, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

        # if (i + 1) % (len(train_loader) // 20) == 0:
        #     print(f"Step {i+1}, Loss: {loss.item()}")
        if i % 400 == 0:
            wandb.log({"loss": loss.item()}, step=i)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Training model...


Epoch 1/1: 100%|██████████| 4820/4820 [03:20<00:00, 24.05it/s]

Epoch 1/1, Loss: 0.0010255926754325628


### Test model on .code only


In [5]:
# Test model on code only
dataset.use_code_only = True
print(f"{dataset.use_code_only=}")
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    arch_stats = {}
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        output = model(batch_x / 255.0)
        batch_y_endian = [endianness_map[e] for e in batch_y["endianness"]]
        batch_y_arch = batch_y["architecture"]
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0

            current_arch = batch_y_arch[i]
            if current_arch not in arch_stats:
                arch_stats[current_arch] = {"correct": 0, "total": 0}

            if pred == batch_y_endian[i]:
                correct += 1
                arch_stats[current_arch]["correct"] += 1

            arch_stats[current_arch]["total"] += 1
            total += 1

    overall_accuracy = correct / total
    print(f"\nOverall Accuracy: {overall_accuracy:.4f} ({correct}/{total})")
    wandb.log({".code_accuracy": overall_accuracy})

    # Print per-architecture accuracies
    print("\nPer-Architecture Accuracies:")
    for arch in sorted(arch_stats.keys()):
        arch_correct = arch_stats[arch]["correct"]
        arch_total = arch_stats[arch]["total"]
        arch_accuracy = arch_correct / arch_total
        arch_stats[arch]["accuracy"] = arch_accuracy
        print(f"{arch:10s}: {arch_accuracy:.4f} ({arch_correct}/{arch_total})")
    wandb.log(arch_stats)
    acc_table = wandb.Table(
        columns=["Architecture", "Accuracy"],
        data=[[arch, arch_stats[arch]["accuracy"]] for arch in arch_stats],
    )
    acc_bar_chart = wandb.plot.bar(
        table=acc_table,
        label="Architecture",
        value="Accuracy",
        title="Accuracy per Architecture",
    )
    wandb.log({"accuracy_bar": acc_bar_chart})
    wandb.run

dataset.use_code_only=True

Overall Accuracy: 0.9717 (18733/19279)

Per-Architecture Accuracies:
alpha     : 0.9976 (826/828)
amd64     : 1.0000 (875/875)
arm64     : 0.9961 (759/762)
armel     : 1.0000 (761/761)
armhf     : 1.0000 (797/797)
hppa      : 0.9936 (927/933)
i386      : 1.0000 (997/997)
ia64      : 1.0000 (1029/1029)
m68k      : 0.9641 (806/836)
mips      : 0.4228 (304/719)
mips64el  : 1.0000 (880/880)
mipsel    : 0.9539 (704/738)
powerpc   : 1.0000 (711/711)
powerpcspe: 0.9987 (765/766)
ppc64     : 0.9914 (575/580)
ppc64el   : 1.0000 (713/713)
riscv64   : 1.0000 (869/869)
s390      : 0.9813 (998/1017)
s390x     : 0.9934 (749/754)
sh4       : 1.0000 (1187/1187)
sparc     : 0.9858 (1045/1060)
sparc64   : 0.9839 (611/621)
x32       : 0.9988 (845/846)


In [6]:
from src.dataset_loaders import MipsMipselDataset as MD


dataset_mips = MD(
    mips_dir="../../dataset/ISAdetect/ISAdetect_full_dataset/mips",
    mipsel_dir="../../dataset/ISAdetect/ISAdetect_full_dataset/mipsel",
    transform=GrayScaleImage(100, 100),
)

train_set, test_set = random_train_test_split(dataset, test_split=0.2, seed=SEED)

mips_dataloader = DataLoader(dataset=dataset_mips, batch_size=4, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    correct_class_1 = 0
    class_1 = 0
    total = 0
    arch_stats = {}
    for batch_x, batch_y in mips_dataloader:
        batch_x = batch_x.to(device)
        output = model(batch_x / 255.0)
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0
            if pred == batch_y[i]:
                correct += 1
            if batch_y[i] == 0:
                class_1 += 1
                if pred == 0:
                    correct_class_1 += 1
            total += 1

    print(f"Accuracy: {correct/total}")
    print(
        f"Class 1: {class_1}, Total: {total}, Percentage: {class_1/total}, Correct: {correct_class_1}"
    )
    wandb.finish()

Accuracy: 0.3013623978201635
Class 1: 3565, Total: 7340, Percentage: 0.48569482288828336, Correct: 2049


.code_accuracy,▁
loss,█▃▂▁▁▁▁▁▁▁▁▁▁
.code_accuracy,0.97168
loss,0.00107
